# General Setup #
First, we are going to load the libraries necessary to run the calculations.

In [ ]:
using CCNO

using ITensors
using ITensorMPS
using Plots
using Measures
using LinearAlgebra
using DelimitedFiles
using Statistics
using Random
using HDF5

Then we are going to define all of the parameters that describe the calculation. These all go into a single data object so the parameters can be passed around easily.

In [ ]:
N_sites_eachflavor= 5 # total sites/particles that evenly spaced "for each (electron) flavor" 
L = 1.0 # cm # domain size # (aka big box length)
Δx = L/N_sites_eachflavor # length of the box of interacting neutrinos at a site in cm  #variable

params = CCNO.Parameters(
    N_sites = 2* (N_sites_eachflavor),
    τ = 5E-13,
    ttotal = 9.0E-11, 
    tolerance  = 5E-1,
    m1 = 0.0,
    m2 = 0.0,
    maxdim = 1,
    cutoff = 1e-100,
    theta_nu = 1.74532925E-8,
    shape_name = "triangular",
    geometric_name = "physical",
    Δx = Δx,
    L = L,
    Δp = Δx,
    periodic = true,
    checkpoint_every = 20,
    do_recover = false,
    recover_file = "",
    plotdir = joinpath(@__DIR__, "plots"),
    datadir = joinpath(@__DIR__,"datafiles"),
    chkptdir = joinpath(@__DIR__, "checkpoints"),
    save_plots_flag = false,
    α = 1e-6
)

# Initial Conditions #
First, we are going to define the physical neutrino densities and energies. These are used to define a list of values of N, which describes the number of physical neutrinos each site represents.

In [ ]:
Δm² = (params.m2^2-params.m1^2) # mass square difference # (erg^2)
n_νₑ =  4.891290848285061e+32 # cm^-3 # number density of electron flavor neutrino
n_νₑ̄ =  n_νₑ # cm^-3 # number density of electron flavor antineutrino
Eνₑ =  50.0*CCNO.MeV # energy of all neutrinos (P.S the its negative is energy of all antineutrinos)
Eνₑ̄ = -1 * Eνₑ # specific to my case only. Since all neutrinos have same energy, except in my case anti neutrinos are moving in opposite direction to give it a negative sign

N = CCNO.Neutrino_number(params, n_νₑ,n_νₑ̄)

Then we set the initial positions of each particle. CCNO is not currently written for more than 1D, so the y and z positions can be ignored. The first half of the particles are distributed through the domain. The second half are also distributed, initially at the same positions as the first half, though this will change as the particles evolve.

In [ ]:
x = CCNO.generate_x_array(N_sites_eachflavor, L)
y = CCNO.generate_x_array(N_sites_eachflavor, L)
z = CCNO.generate_x_array(N_sites_eachflavor, L)

Set the momenta of the particles. The first half of the particles have a positive x momentum and the second half have a negative x momentum.

In [ ]:
p = hcat(CCNO.generate_px_array(params.N_sites, Eνₑ, Eνₑ̄), CCNO.generate_py_array(params.N_sites), CCNO.generate_pz_array(params.N_sites))

Setting the "energy sign", which is a tool to be able to replicate some tests in the literature. Since we are not using that tool, we just set all values to 1.

In [ ]:
energy_sign = [1 for i in 1:params.N_sites]

Generate a list of unique site indices that will remain attached to the particles throughout their evolution. We specify that the sites represent spin-1/2 (i.e., two-level) states for each particle.

In [ ]:
s = siteinds("S=1/2", params.N_sites; conserve_qns=false) 

Set up the initial quantum state. The first half of the particles are "spin up" (i.e., electron neutrinos, correpsonding to the right-moving states). The second half are "spin down" (i.e., muon neutrinos, corresponding to the left-moving states).

In [ ]:
ψ = productMPS(s, n -> n <= params.N_sites/2 ? "Up" : "Dn")

Bundle all of the state variables into a single mutable object that will be modified during the evolution loop.

In [ ]:
state = CCNO.SimulationState(ψ=ψ, s=s, p=p, energy_sign = energy_sign, N=N, xyz = hcat(x,y,z))

Perturb the initial conditions slightly with a wavelength equal to the domain size in order to tease out plasma-like instabilities.

In [ ]:
k = 2*pi / (L)
CCNO.perturb(params, state,k, params.theta_nu)

Remove old data files and evolve the quantum state. Caution: the first two lines delete data - be careful about preserving data you want to keep.

In [ ]:
rm("checkpoints", recursive=true, force=true)
rm("datafiles", recursive=true, force=true)
CCNO.evolve(params, state)

# Data Analysis #
Read the output data files and store in local arrays

In [ ]:
t_Sz_tot = readdlm(joinpath(params.datadir, "t_<Sz>.dat"))
t_Sy_tot = readdlm(joinpath(params.datadir, "t_<Sy>.dat"))
t_Sx_tot = readdlm(joinpath(params.datadir, "t_<Sx>.dat"))
t_xsiteval = readdlm(joinpath(params.datadir, "t_xsiteval.dat"))
t_pxsiteval = readdlm(joinpath(params.datadir, "t_pxsiteval.dat"))
t_ρₑₑ_tot = readdlm(joinpath(params.datadir, "t_ρₑₑ.dat"))
t_ρ_μμ_tot = readdlm(joinpath(params.datadir, "t_ρ_μμ.dat"))
t_ρₑμ_tot = readdlm(joinpath(params.datadir, "t_ρₑμ.dat"))

Put the data into usable arrays. The first column of every file contains the time. Make an array with the time, and an array with all of the other useful values.

In [ ]:
t_array = t_Sz_tot[:, 1]  
Sz_array =  t_Sz_tot[:, 2:N_sites_eachflavor+1]
Sy_array = t_Sy_tot[:, 2:N_sites_eachflavor+1]  
Sx_array= t_Sx_tot[:, 2:N_sites_eachflavor+1] 
ρₑₑ_array = t_ρₑₑ_tot[:, 2:N_sites_eachflavor+1] 
ρ_μμ_array =t_ρ_μμ_tot[:, 2:N_sites_eachflavor+1]  
ρₑμ_array = t_ρₑμ_tot[:, 2:N_sites_eachflavor+1]

Make some preliminary plots. First, show the evolution of the off-diagonal elements averaged over the domain.

In [ ]:
using Plots

ρₑμ_array_domain_avg = mean(abs.(ρₑμ_array), dims=2)
plot(t_array, ρₑμ_array_domain_avg, xlabel="t (s)", ylabel="|ρₑμ|", yscale=:log10, label="")